In [4]:
!pip install biopython

from Bio import SeqIO, Align
import numpy as np

# Read one sequence from each FASTA file
with open('BVBRC_genome_sequence.fasta', 'r') as f:
    seq1 = str(next(SeqIO.parse(f, 'fasta')).seq)[:100]

with open('BVBRC_genome_sequence_HOS52.fasta', 'r') as f:
    seq2 = str(next(SeqIO.parse(f, 'fasta')).seq)[:100]

# 1. BIOPYTHON IMPLEMENTATION
print("=" * 50)
print("BIOPYTHON IMPLEMENTATION")
print("=" * 50)

aligner = Align.PairwiseAligner()
aligner.mode = 'global'
aligner.match_score = 1
aligner.mismatch_score = -1
aligner.gap_score = -2

alignments = aligner.align(seq1, seq2)
for alignment in alignments:
    print(alignment)
    print(f"Score: {alignment.score}")
    break

# 2. MANUAL IMPLEMENTATION
print("\n" + "=" * 50)
print("MANUAL IMPLEMENTATION")
print("=" * 50)

def needleman_wunsch(seq1, seq2, match=1, mismatch=-1, gap=-2):
    n = len(seq1)
    m = len(seq2)

    # Initialize matrix
    score_matrix = np.zeros((n + 1, m + 1))

    for i in range(n + 1):
        score_matrix[i][0] = gap * i
    for j in range(m + 1):
        score_matrix[0][j] = gap * j

    # Fill matrix
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            if seq1[i-1] == seq2[j-1]:
                diagonal = score_matrix[i-1][j-1] + match
            else:
                diagonal = score_matrix[i-1][j-1] + mismatch

            up = score_matrix[i-1][j] + gap
            left = score_matrix[i][j-1] + gap

            score_matrix[i][j] = max(diagonal, up, left)

    # Traceback
    align1 = ""
    align2 = ""
    i = n
    j = m

    while i > 0 or j > 0:
        current = score_matrix[i][j]

        if i > 0 and j > 0:
            diagonal = score_matrix[i-1][j-1] + (match if seq1[i-1] == seq2[j-1] else mismatch)
        else:
            diagonal = float('-inf')

        up = score_matrix[i-1][j] + gap if i > 0 else float('-inf')
        left = score_matrix[i][j-1] + gap if j > 0 else float('-inf')

        if i > 0 and j > 0 and current == diagonal:
            align1 = seq1[i-1] + align1
            align2 = seq2[j-1] + align2
            i -= 1
            j -= 1
        elif i > 0 and current == up:
            align1 = seq1[i-1] + align1
            align2 = "-" + align2
            i -= 1
        else:
            align1 = "-" + align1
            align2 = seq2[j-1] + align2
            j -= 1

    return align1, align2, score_matrix[n][m]

align1, align2, score = needleman_wunsch(seq1, seq2)

print("Alignment:")
print(align1)
print(align2)
print(f"\nScore: {score}")

BIOPYTHON IMPLEMENTATION
target            0 c-caacgacgaccca-ccgccgttttgatgtggaacggcctgtc-aagtgg--ccga-tt
                  0 .-|.|.|.||||.||-||.||....-|||..|..|-|....|.|-||.|.|--||||-||
query             0 tgccaggccgacacaaccacccaca-gatcagtca-gtagagcccaattcgtaccgaatt

target           54 agtg--cttgttgcctcggggttgtttggggtttctggctttgatccg- 100
                 60 .|.|--|||-||.|.||----|-|.|.|.|.|..|..|||.-||.|.|- 109
query            58 tgggggctt-ttacgtc----t-gctcgcgctacccagcta-gaccggg 100

Score: -17.0

MANUAL IMPLEMENTATION
Alignment:
-ccaacgacgac-ccaccgccgttttgat--gt-ggaacggcctgtcaagtggccg-a-tt-agtgcttgttgcctcggggttgtttggggtttctggctttgatc-cg
tgccaggccgacacaaccacc-cacagatcagtcagtagagcccaattcgt-accgaatttgggggctt-ttacgtc-----tgctcgcgctacccagc-tagaccggg

Score: -17.0
